To run, need to connect to fatboy:
 ssh -L 51433:fatboy.phys.washington.edu:1433 gateway.astro.washington.edu

If non-astro user, use simsuser@gateway.astro.washington.edu


In [1]:
%matplotlib inline
import numpy as np
import healpy as hp
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic
from lsst.sims.catalogs.db import CatalogDBObject
from lsst.sims.catUtils.baseCatalogModels import *
from lsst.sims.catUtils.exampleCatalogDefinitions import *
from lsst.sims.utils import ObservationMetaData

/Users/yoachim/lsst/DarwinX86/miniconda2/3.19.0.lsst4/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/yoachim/gitRepos/sims_catalogs/python/lsst/sims/catalogs/db/dbConnection.py:439: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/Users/yoachim/gitRepos/sims_catalogs/python/lsst/sims/catalogs/db/dbConnection.py:439: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


In [2]:
# Coordinates at the galactic pole
c = SkyCoord(Galactic, l=0.*u.degree , b=-90*u.degree)

In [3]:
c.icrs.ra.deg, c.icrs.dec

(12.85947789477605, <Latitude -27.128252414968006 deg>)

In [4]:
# Radius to query, in degrees
boundLength = 3.5/2.

colnames = ['raJ2000', 'decJ2000', 'umag', 'gmag', 'rmag', 'imag', 'zmag', 'ymag']
constraint = 'rmag < 16'

# dbobj = CatalogDBObject.from_objid('allstars')
dbobj = CatalogDBObject.from_objid('brightstars')

obs_metadata = ObservationMetaData(boundType='circle',
                                   pointingRA=c.icrs.ra.deg,
                                   pointingDec=c.icrs.dec.deg,
                                   boundLength=boundLength, mjd=5700)

t = dbobj.getCatalog('ref_catalog_star', obs_metadata=obs_metadata)

stars = t.db_obj.query_columns(colnames=colnames, obs_metadata=obs_metadata,
                               constraint=constraint, limit=1e6, chunk_size=None)
stars = [chunk for chunk in stars][0]


/Users/yoachim/lsst/DarwinX86/sqlalchemy/1.0.8.lsst3+2/lib/python/SQLAlchemy-1.0.8-py2.7-macosx-10.5-x86_64.egg/sqlalchemy/sql/elements.py:3779: SAWarning: Textual SQL expression 'rmag < 16' should be explicitly declared as text('rmag < 16') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})


In [5]:
stars

rec.array([ (225426217, 0.2089296122983494, -0.48961108895629474, 17.77929, 15.12636, 13.90252, 13.44415, 13.22605, 13.21823),
 (225426206, 0.2072614091460007, -0.489929890797464, 13.76424, 11.50369, 10.46925, 10.06904, 9.892447, 9.838599),
 (225426212, 0.20805576084846086, -0.4890868793154583, 18.36475, 15.68047, 14.44254, 13.99018, 13.77389, 13.78084),
 ...,
 (242655789, 0.21582036417144074, -0.44799139165458485, 16.94425, 14.36904, 13.17815, 12.71532, 12.49625, 12.46306),
 (242655779, 0.21483554468771043, -0.44798240320893706, 17.17073, 14.5457, 13.33491, 12.87094, 12.65116, 12.62986),
 (242655778, 0.2148122619954888, -0.4481498500973734, 16.94808, 14.42564, 13.2607, 12.8041, 12.59009, 12.54844)], 
          dtype=[('simobjid', '<i8'), ('raJ2000', '<f8'), ('decJ2000', '<f8'), ('umag', '<f8'), ('gmag', '<f8'), ('rmag', '<f8'), ('imag', '<f8'), ('zmag', '<f8'), ('ymag', '<f8')])

In [16]:
# Parameters we need to set

# Magnitudes of gray cloud extinction one should be able to see through
max_extinction = 1.0

# Precision of the measured extinction
extinction_precision = 0.2

# Dynamic range of all sky camera
mag_range = [7, 12]

# Angular resolution requirement, diameter
ang_resolution = 3. #  Degrees

# Time resolution
time_resolution = 30. #  Seconds

# Seeing requirement

# Sky brightness requirement

# Airmass requrement
airmass_limit = 2.0

# Atmospheric extinction
kAtm = 0.15

In [17]:
# Zeropoint uncertainty is weighted mean of zeropoints computed from each star.  
# Should be able to adjust 5-sigma depth to get to extinction precision required. 